# Extrair apenas eventos relacionados a votos e metadados

Como o Log das urnas compila todo o histórico de eventos que ocorreu em uma unna, incluindo testes, carga, preparação, etc, para garantir uma consulta mais eficiente, são extraídos apenas os eventos relacionados aos votos em si.



## Importing libraries

In [6]:
import duckdb
import pandas as pd
import time

## Importing Data

In [7]:
DATASET = 'ALL_UFS.parquet'
COLUMN_EVENT_DESCRIPTION = 'event_description'

In [18]:
cursor = duckdb.connect()

## Preaparing Data

### Definindo os filtros

In [44]:
METADATA = [
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Zona Eleitoral%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Turno da UE%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Município%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Identificação do Modelo de Urna%'"
]

EVENTS_DESCRIPTIONS = [
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Urna pronta para receber voto%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Voto confirmado para%'"
]

VOTES_DESCRIPTIONS = [
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor foi habilitado%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor foi suspenso e não confirmou nenhum voto%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor foi suspenso pelo mesário%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor impedido%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor já justificou%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor não está votando%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor não tem idade mínima%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor sem atividade por%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor voltou a digitar enquanto estava sendo suspenso%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'Eleitor votou parcialmente e em seguida foi suspenso%'",
    F"{COLUMN_EVENT_DESCRIPTION} ILIKE 'O voto do eleitor foi computado%'"
]

ACCEPTED_DATES = [
    '2022-10-03', '2022-10-30', # Constitutional Election Dates
    '2022-10-04', '2022-10-31', # In case a section lasts until the next day
]

ALL_FILTERS = METADATA + EVENTS_DESCRIPTIONS + VOTES_DESCRIPTIONS

### Construindo e executando a query

In [45]:
query = F"""
    SELECT
        event_timestamp,
        event_timestamp::date AS event_date,
        event_type,
        some_id,
        event_system,
        event_description,
        event_id,
        -- filename,

        REPLACE(SPLIT_PART(filename, '/', 4), '2_', '') AS uf,
        REPLACE(SPLIT_PART(filename, '/', 5), '_new.csv', '') AS filename
    FROM
        {DATASET}
    WHERE 1=1 AND
        ( {' OR '.join(ALL_FILTERS)} )
        AND event_timestamp::date IN ({', '.join([F"'{date}'" for date in ACCEPTED_DATES])})
"""

In [ ]:
query = F"""
    COPY ({query}) TO 'UFS_VOTES.parquet' (FORMAT 'parquet', PARTITION_BY (event_date, uf))
"""

In [ ]:
tic = time.time()
cursor.execute(query)
toc = time.time()

print(F"Query executed in {toc - tic:.2f} seconds")
